# Lab | Customer Analysis Round 6
For this lab, we still keep using the `marketing_customer_analysis.csv` file that you can find in the `files_for_lab` folder.

### Get the data
We are using the `marketing_customer_analysis.csv` file.

In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import norm
from scipy import stats #para box-cox entre otros
from scipy.stats import skew
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

### Dealing with the data

Already done in the round 2.

In [113]:
# Cargamos el archivo ya limpio

df= pd.read_csv('marketing_customer_analysis(2).csv')

### EDA (Exploratory Data Analysis) - Complete the following 

Done in the round 3.

In [114]:
df_2= df.copy()
df_2.columns

Index(['unnamed', 'customer', 'state', 'customer lifetime value', 'response',
       'coverage', 'education', 'effective to date', 'employmentstatus',
       'gender', 'income', 'location code', 'marital status',
       'monthly premium auto', 'months since last claim',
       'months since policy inception', 'number of open complaints',
       'number of policies', 'policy type', 'policy', 'renew offer type',
       'sales channel', 'total claim amount', 'vehicle class', 'vehicle size',
       'vehicle type', 'Month'],
      dtype='object')

In [115]:
df_2.shape

(10910, 27)

In [116]:
# Renombrar la columna 'unnamed: 0' a 'unnamed'
df_2.rename(columns={'unnamed: 0': 'unnamed'}, inplace=True)
df_2.info

<bound method DataFrame.info of        unnamed customer       state  customer lifetime value response  \
0            0  DK49336     Arizona              4809.216960       No   
1            1  KX64629  California              2228.525238       No   
2            2  LZ68649  Washington             14947.917300       No   
3            3  XL78013      Oregon             22332.439460      Yes   
4            4  QA50777      Oregon              9025.067525       No   
...        ...      ...         ...                      ...      ...   
10905    10905  FE99816      Nevada             15563.369440       No   
10906    10906  KX53892      Oregon              5259.444853       No   
10907    10907  TL39050     Arizona             23893.304100       No   
10908    10908  WA60547  California             11971.977650       No   
10909    10909  IV32877      Unknow              6857.519928   Unknow   

       coverage education effective to date employmentstatus gender  ...  \
0         Basic

In [117]:
# Visualizamos la columnas que son numericas
df_2._get_numeric_data()

,unnamed,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1
...,...,...,...,...,...,...,...,...,...,...
10905,10905,15563.369440,0,253,15.149071,40,0.384256,7,1214.400000,1
10906,10906,5259.444853,61146,65,7.000000,68,0.000000,6,273.018929,1
10907,10907,23893.304100,39837,201,11.000000,63,0.000000,2,381.306996,2
10908,10908,11971.977650,64195,158,0.000000,27,4.000000,6,618.288849,2


In [118]:
df_2.describe

<bound method NDFrame.describe of        unnamed customer       state  customer lifetime value response  \
0            0  DK49336     Arizona              4809.216960       No   
1            1  KX64629  California              2228.525238       No   
2            2  LZ68649  Washington             14947.917300       No   
3            3  XL78013      Oregon             22332.439460      Yes   
4            4  QA50777      Oregon              9025.067525       No   
...        ...      ...         ...                      ...      ...   
10905    10905  FE99816      Nevada             15563.369440       No   
10906    10906  KX53892      Oregon              5259.444853       No   
10907    10907  TL39050     Arizona             23893.304100       No   
10908    10908  WA60547  California             11971.977650       No   
10909    10909  IV32877      Unknow              6857.519928   Unknow   

       coverage education effective to date employmentstatus gender  ...  \
0         Bas

In [119]:
# Seleccionar todas las columnas categóricas
df_2_object= df_2.select_dtypes(object)
df_2_numerical= df_2.select_dtypes(np.number)

1.7 DATA PREPROCESSING

In [120]:
df_2_numerical

,unnamed,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1
...,...,...,...,...,...,...,...,...,...,...
10905,10905,15563.369440,0,253,15.149071,40,0.384256,7,1214.400000,1
10906,10906,5259.444853,61146,65,7.000000,68,0.000000,6,273.018929,1
10907,10907,23893.304100,39837,201,11.000000,63,0.000000,2,381.306996,2
10908,10908,11971.977650,64195,158,0.000000,27,4.000000,6,618.288849,2


In [121]:
df_2_numerical.head()

,unnamed,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1


In [122]:
df_2_num= df_2_numerical.drop('unnamed', axis= 1)

In [123]:
df_2_num.shape

(10910, 9)

In [124]:
#OLS Regression Results  

Y=df_2_num['total claim amount']
X=df_2_num.drop(['total claim amount'], axis=1)
x=sm.add_constant(X)
model=sm.OLS(Y,X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:     total claim amount   R-squared (uncentered):                   0.850
Model:                            OLS   Adj. R-squared (uncentered):              0.850
Method:                 Least Squares   F-statistic:                              7731.
Date:                Thu, 18 May 2023   Prob (F-statistic):                        0.00
Time:                        18:10:44   Log-Likelihood:                         -73437.
No. Observations:               10910   AIC:                                  1.469e+05
Df Residuals:                   10902   BIC:                                  1.469e+05
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

In [125]:
#Normalizando 

transformer=StandardScaler().fit(df_2_num)
x_normalized=transformer.transform(df_2_num)
print(x_normalized.shape)
df_numer= pd.DataFrame(x_normalized)

(10910, 9)


In [126]:
#One Hot/Label Encoding (categorical).

df_2_object2= df_2_object.drop(['effective to date','customer'], axis=1)
print(df_2_object2)


            state response  coverage education employmentstatus gender  \
0         Arizona       No     Basic   College         Employed      M   
1      California       No     Basic   College       Unemployed      F   
2      Washington       No     Basic  Bachelor         Employed      M   
3          Oregon      Yes  Extended   College         Employed      M   
4          Oregon       No   Premium  Bachelor    Medical Leave      F   
...           ...      ...       ...       ...              ...    ...   
10905      Nevada       No   Premium  Bachelor       Unemployed      F   
10906      Oregon       No     Basic   College         Employed      F   
10907     Arizona       No  Extended  Bachelor         Employed      F   
10908  California       No   Premium   College         Employed      F   
10909      Unknow   Unknow     Basic  Bachelor       Unemployed      M   

      location code marital status     policy type        policy  \
0          Suburban        Married  Corpora

In [127]:
#para sacar dummies vamos a usar la liberería OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(handle_unknown='error', drop="first").fit(df_2_object2)
encoder.categories_
encoded=encoder.transform(df_2_object2).toarray()
encoded

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [128]:
datatest= df_2_object2.copy()

# Create an instance of the OneHotEncoder with drop='first' to drop the first column of each dummy variable
encoder = OneHotEncoder(handle_unknown='error', drop='first')

# Fit the encoder to the categorical data
encoder.fit(df_2_object2)

# Transform the categorical data into dummy variables
encoded = encoder.transform(df_2_object2).toarray()

# Get the feature names of the encoded variables
feature_names = encoder.get_feature_names_out(df_2_object2.columns)

# Convert the encoded array to a DataFrame with column names
X_cat_encoded = pd.DataFrame(encoded, columns=feature_names)

# Drop the original categorical columns from the original DataFrame
datatest = datatest.drop(df_2_object2.columns, axis=1)

# Concatenate the encoded DataFrame with the original DataFrame
datatest = pd.concat([datatest, X_cat_encoded], axis=1)

# Visualization of code
datatest

,state_California,state_Nevada,state_Oregon,state_Unknow,state_Washington,response_Unknow,response_Yes,coverage_Extended,coverage_Premium,education_College,...,vehicle class_Luxury Car,vehicle class_Luxury SUV,vehicle class_SUV,vehicle class_Sports Car,vehicle class_Two-Door Car,vehicle class_Unknow,vehicle size_Medsize,vehicle size_Small,vehicle size_Unknow,vehicle type_Unknow
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10906,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
10908,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [129]:
concatenated_df = pd.concat([df_2_num, datatest], axis=1)

In [130]:
concatenated_df

,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month,state_California,...,vehicle class_Luxury Car,vehicle class_Luxury SUV,vehicle class_SUV,vehicle class_Sports Car,vehicle class_Two-Door Car,vehicle class_Unknow,vehicle size_Medsize,vehicle size_Small,vehicle size_Unknow,vehicle type_Unknow
0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,15563.369440,0,253,15.149071,40,0.384256,7,1214.400000,1,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10906,5259.444853,61146,65,7.000000,68,0.000000,6,273.018929,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10907,23893.304100,39837,201,11.000000,63,0.000000,2,381.306996,2,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
10908,11971.977650,64195,158,0.000000,27,4.000000,6,618.288849,2,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [131]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score

In [132]:
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.2, random_state=42)
lm=linear_model.LinearRegression()
model=lm.fit(X_train, y_train)
predictions=lm.predict(X_test)
r2_score(y_test, predictions)

0.5069409388151643

In [133]:
dfpred=pd.DataFrame(columns=[y_test, predictions]).T
dfpred

,
total claim amount,
475.423848,123.015467
350.400000,350.476172
482.400000,415.954144
673.342650,435.205053
302.400000,154.049526
...,...
432.000000,430.769860
484.800000,561.515749
525.600000,328.107068


In [134]:
#Lección 3: Interpretar resultados de un train-test de un modelo de regresión Lineal (no liarse con Clasificación)
#MSE, RMSE, R2, ADJ R2

import math
from sklearn.metrics import mean_squared_error

mse=mean_squared_error(y_test, predictions) #varianza
print(mse)
rmse= math.sqrt(mse) #error estándar
print(rmse)

r2=r2_score(y_test, predictions) # valores de 0 a 1, cuanto más cercano a 1 mejor. 
print(r2)

n=len(X_test) #igual que r2 pero penaliza el incluir variables independientes que no aporten al modelo. 
p=X_test.shape[1]
adj_r2=1-((1-r2)*(n-1)/(n-p-1))
print(adj_r2)

39811.05540482342
199.52707937727004
0.5069409388151643
0.5051257190777144
